In [1]:
import pandas as pd
import numpy as np 
import os, sys
sys.path.append('/home/jessica/labelGAN/StyleGAN.pytorch/')
from models.GAN import * 
from stylegan_layers import  G_mapping,G_synthesis
from sklearn.neighbors import KernelDensity
from torchvision.utils import save_image
import torch

In [2]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
g_all = nn.Sequential(OrderedDict([('g_mapping', G_mapping()),
    #('truncation', Truncation(avg_latent)),
    ('g_synthesis', G_synthesis(resolution=1024))    
    ]))

opts = {'mapping_layers': 8, 'truncation_psi': -1.}
'''g_all = Generator(resolution=1024,
                    num_channels=3,
                    structure='linear',
                    **opts)'''
g_all = nn.Sequential(OrderedDict([('g_mapping', GMapping()),
    #('truncation', Truncation(avg_latent)),
    ('g_synthesis', GSynthesis(resolution=1024, depth=8))    
    ]))
#Load the pre-trained model
g_all.load_state_dict(torch.load('/data1/jessica/data/labelGAN/checkpoints/styleGAN/GAN_GEN_8.pth', map_location=device), strict=False)
g_all.eval()
g_all.to(device)
g_mapping, g_synthesis = g_all[0], g_all[1]

In [3]:
image_latents_names = [elem.replace("latent_", "").replace(".npy", "") for elem in os.listdir('/home/jessica/labelGAN/Image2StyleGAN/images/latents')]
df = pd.read_csv("vinbig_25_samples.csv")
df_sampled = df[df.image_id.isin(image_latents_names)]
kernel = 'exponential'
for class_name in df.class_name.unique():
   class_name = class_name.replace("/", "").replace(" ", "")
   test = df_sampled[df_sampled.class_name == class_name].image_id
   images = test.apply(lambda x: np.load("./images/latents/latent_" + x + ".npy"))
   if len(images) == 0: continue
   t = np.concatenate(np.array(images), axis=0)
   t = t.reshape(t.shape[0], -1)# create an array of samples
   samples = np.array(t)

   # fit a kernel density estimator to the samples
   kde = KernelDensity(bandwidth=0.5, kernel=kernel)
   kde.fit(samples)

   # sample from the kde
   new_samples = kde.sample(100)
   for i, latents in enumerate(new_samples):
      latents = torch.tensor(latents.reshape(1, 18, 512)).to("cuda:2").float()
      syn_img = g_synthesis(latents, depth=8)
      syn_img = (syn_img+1.0)/2.0
      save_image(syn_img.clamp(0,1),f"./images/generated_latents_from_class_distr/{class_name}_{i}_{kernel}.png")